In [15]:
# !pip install bert-embedding
# !pip install mxnet-cu80

UNCOMMENT THESE AND RUN

THEN AFTER THAT COMMENT THESE AGAIN AND RESTART THE KERNEL AND RUN ALL

In [16]:
import tensorflow as tf
# import cv2 as cv
# import os
import numpy as np
import glob
# from random import shuffle
# from google.colab import drive
import matplotlib.pyplot as plt
import csv
import json
from bert_embedding import BertEmbedding
import mxnet as mx
import re


In [17]:
# drive.mount("/content/drive")

In [18]:
def clean_text(text):
    text = text.lower()
    text = re.sub(r"i'm","i am",text)
    text = re.sub(r"i've","i have",text)
    text = re.sub(r"he's","he is",text)
    text = re.sub(r"she's","she is",text)
    text = re.sub(r"that's","that is",text)
    text = re.sub(r"that ' s","that is",text)
    text = re.sub(r"it's","it is",text)
    text = re.sub(r"that's","that is",text)
    text = re.sub(r"where's","where is",text)
    text = re.sub(r"what's","what is",text)
    text = re.sub(r"\'ll"," will",text) 
    text = re.sub(r"\'ve"," have",text)
    text = re.sub(r"\'re"," are",text)
    text = re.sub(r"\'d"," would",text)  
    text = re.sub(r"won't", "will not",text)
    text = re.sub(r"don't", "do not",text)
    text = re.sub(r"can't","can not", text)
    text = re.sub(r"hadn't","had not", text)
    text = re.sub(r"didn't","did not", text)
    text = re.sub(r"wouldn't","would not", text)
    text = re.sub(r"weren't","were not", text)
    text = re.sub(r"shouldn't","should not", text)
    text = re.sub(r"doesn't","does not", text)
    text = re.sub(r"couldn't","could not", text)
    text = re.sub(r"isn't","is not", text)
    text = re.sub(r"hasn't","has not", text)
    text = re.sub(r"wasn't","was not", text)
    text = re.sub(r"haven't","have not", text)
    text = re.sub(r"didn't","did not", text)
    text = re.sub(r"wouldnt'","would not", text)
#     text = re.sub(r"shouldnt'","should not", text)
#     text = re.sub(r"doesnt'","does not", text)
#     text = re.sub(r"couldnt'","could not", text)
#     text = re.sub(r"isnt'","is not", text)
#     text = re.sub(r"wasn't","was not", text)
#     text = re.sub(r"havent'","have not", text)
    text = re.sub(r"aren't","are not", text)
#     text = re.sub(r"werent","were not", text)
    text = re.sub(r" em "," them ", text)
    text = re.sub(r" there's "," there is ", text)
    text = re.sub(r"let's","let us", text)
    text = re.sub(r" who's "," who is ", text)
    text = re.sub(r"\'s","", text)
    text = re.sub(r"'","", text)
# #     text = re.sub(r"arent","are not", text)
#     text = re.sub(r"  "," ", text)
#     text = re.sub(r"[-?!@#$%^&*(~,;)+=_`<>{}.\"']","",text)
#     pattern = r'[^a-zA-z0-9\s]'
#     text = re.sub(pattern, '', text)
    return text

USE THIS FOR 1024 EMBEDDING LENGTH

In [19]:
final_emb={}

In [20]:
def create_embedding(data):
  a=1
  Participant_transcript=[rows[0].split("\t") for rows in data if len(rows)>0 if rows[0].split("\t")[2] == "Participant" ]
#   print(Participant_transcript)
  words=[clean_text(x[3].strip()).strip().split(" ") for x in Participant_transcript]
  while(a):
    a=len([x.remove('') for x in words if '' in x])
#   print(words[95])
  for kk in words:
        for ind,val in enumerate(kk):
            if(val[0]=='<'and val[-1]=='>'):
                kk[ind]=val[1:-1]
            elif(val[0]=='<' ):
                kk[ind]=val[1:]
#   print(words)              
  ctx = mx.gpu(0)
  embedding= BertEmbedding(model='bert_24_1024_16', dataset_name='book_corpus_wiki_en_cased',ctx=ctx)
  emb=[embedding(x) for x in words]
#   cnt=0
  for x in emb:
    for i in range(len(x)):
      if (x[i][0][0].isalpha()) and (x[i][0][0] not in final_emb):
        final_emb[x[i][0][0]]=x[i][1][0].tolist()
#     print("ln",cnt)
#     cnt+=1


CHANGE PATH TO DATASET

In [ ]:
for filename in glob.glob("./trans/*RIPT.csv"):#HERE
    print(filename)
    with open(filename,"rt") as f:
      data=csv.reader(f)
      create_embedding(data)
#       a=1
#       Participant_transcript=[rows[0].split("\t") for rows in data if len(rows)>0 if rows[0].split("\t")[2] == "Participant" ]
#       print(Participant_transcript)
#       words=[clean_text(x[3].strip()).split(" ") for x in Participant_transcript]
#       print(words)
#       while(a):
#         a=len([x.remove('') for x in words if '' in x])
#       #   print(words[95])
#       ctx = mx.gpu(0)
#       embedding= BertEmbedding(model='bert_24_1024_16', dataset_name='book_corpus_wiki_en_cased',ctx=ctx)
#       emb=[embedding(x) for x in words]
# #         cnt=0
#       for x in emb:
#         for i in range(len(x)):
#           if (x[i][0][0].isalpha()) and (x[i][0][0] not in final_emb):
#             final_emb[x[i][0][0]]=x[i][1][0].tolist()

./trans/471_TRANSCRIPT.csv


In [ ]:
with open('./Bert_1024B.json', 'w') as json_file:  
    json.dump(final_emb, json_file)

Change path to save the json for  bert embeddings

In [ ]:
len(final_emb)


In [ ]:
# emb[0][0][1][0].shape#list[list(tuple[list[array
final_emb